In [41]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("AmazonData.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                10002 non-null  object 
 1   Product Name           10002 non-null  object 
 2   Brand Name             0 non-null      float64
 3   Asin                   0 non-null      float64
 4   Category               9172 non-null   object 
 5   Upc Ean Code           34 non-null     object 
 6   List Price             0 non-null      float64
 7   Selling Price          9895 non-null   object 
 8   Quantity               0 non-null      float64
 9   Model Number           8232 non-null   object 
 10  About Product          9729 non-null   object 
 11  Product Specification  8370 non-null   object 
 12  Technical Details      9212 non-null   object 
 13  Shipping Weight        8864 non-null   object 
 14  Product Dimensions     479 non-null    object 
 15  Im

In [42]:
cols = [0,2,3,5,6,8,9,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
dataset.drop(dataset.columns[cols], axis =1, inplace=True)
dataset.dropna(inplace = True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 10001
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product Name           7216 non-null   object
 1   Category               7216 non-null   object
 2   Selling Price          7216 non-null   object
 3   About Product          7216 non-null   object
 4   Product Specification  7216 non-null   object
 5   Shipping Weight        7216 non-null   object
dtypes: object(6)
memory usage: 394.6+ KB


In [43]:
new = dataset["Category"].str.split("|", n = 3, expand = True)
new = new.apply(lambda x: x.str.strip())
  
# making the first category called Main Category
dataset["Main Category"]= new[0] 
  
# making the second category called sub_category 
dataset["Sub Category"]= new[1]

# making the third category called side_category 
dataset["Side Category"]= new[2]

# making the last column consist of the remaining categories
dataset["Other Category"]= new[3]

# Dropping old category columns and the remaining categories 
dataset.drop(columns =["Category"], inplace = True)

In [44]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7216 entries, 0 to 10001
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product Name           7216 non-null   object
 1   Selling Price          7216 non-null   object
 2   About Product          7216 non-null   object
 3   Product Specification  7216 non-null   object
 4   Shipping Weight        7216 non-null   object
 5   Main Category          7216 non-null   object
 6   Sub Category           7216 non-null   object
 7   Side Category          6231 non-null   object
 8   Other Category         2791 non-null   object
dtypes: object(9)
memory usage: 563.8+ KB


In [45]:
product_line = int(input("Digite o número do produto: "))
product_line = product_line - 2

if 0 <= product_line < len(dataset):
    produto_selecionado = dataset.iloc[product_line]  # Seleciona o produto pela linha
    print("\nProduto selecionado:")
    print(produto_selecionado)
else:
    print("Número de linha inválido. Tente novamente.")

main_category_input = produto_selecionado['Main Category']
sub_category_input = produto_selecionado['Sub Category']
side_category_input = produto_selecionado['Side Category']
other_category_input = produto_selecionado['Other Category']

print("\n")
print(main_category_input)
print(sub_category_input)
print(side_category_input)
print(other_category_input)



Produto selecionado:
Product Name             Rubie's Child's Pokemon Deluxe Pikachu Costume...
Selling Price                                                       $29.12
About Product            Make sure this fits by entering your model num...
Product Specification    ProductDimensions:16x8x1inches|ItemWeight:1pou...
Shipping Weight                                                 9.8 ounces
Main Category                                    Clothing, Shoes & Jewelry
Sub Category                                        Costumes & Accessories
Side Category                                                  Kids & Baby
Other Category                                            Girls | Costumes
Name: 6, dtype: object


Clothing, Shoes & Jewelry
Costumes & Accessories
Kids & Baby
Girls | Costumes


In [46]:
# Verifica se as categorias fornecidas são válidas
if main_category_input not in dataset['Main Category'].unique():
    print('Main Category não encontrada na base de dados.')
if sub_category_input not in dataset['Sub Category'].unique():
    print('Subcategoria não encontrada na base de dados.')
if side_category_input not in dataset['Side Category'].unique():
    print('Side Category não encontrada na base de dados.')
if other_category_input not in dataset['Other Category'].unique():
    print('Other Category não encontrada na base de dados.')

In [47]:

# Calcula uma pontuação para cada produto com base na similaridade das categorias
dataset['score'] = 0

# Aumenta a pontuação se a "Main Category" ou "Sub Category" corresponder

dataset.loc[dataset['Main Category'] == main_category_input, 'score'] += 1
dataset.loc[dataset['Sub Category'] == sub_category_input, 'score'] += 1
dataset.loc[dataset['Side Category'] == side_category_input, 'score'] += 1
dataset.loc[dataset['Other Category'] == other_category_input, 'score'] += 1


In [48]:

# Ordena os produtos com base na pontuação, do mais alto para o mais baixo
produtos_recomendados = dataset.sort_values(by='score', ascending=False)

# Remove produtos com pontuação 0 (sem correspondência)
produtos_recomendados = produtos_recomendados[produtos_recomendados['score'] > 0]

# Remove a coluna de pontuação antes de retornar
produtos_recomendados = produtos_recomendados.drop(columns='score')

# Retorna as recomendações
if produtos_recomendados.empty:
    print('Nenhuma recomendação encontrada para essas categorias.')

print(produtos_recomendados)

                                           Product Name Selling Price  \
7359                    Medieval Princess Girls Costume        $27.24   
8901  Rubie's Costume Girls DC Superhero Deluxe Star...        $16.43   
8744  Princess Paradise Skele-Bat Child's Costume, 1...        $31.92   
2397                          Cool Ghoul Costume, Large        $12.61   
1493   Disney Minnie Mouse Toddler Girls' Costume, Pink        $17.46   
...                                                 ...           ...   
5764  Marvel Avengers Assemble Captain America Costu...        $18.75   
4420  Great Eastern Entertainment Hellsing OVA Penta...         $7.14   
7982                      Batgirl Glitter Slipper Shoes         $9.32   
8728  The Riddler-Poison DC Comics Originals Ivy Str...        $11.99   
6443  Great Eastern Entertainment Negima School Logo...         $3.55   

                                          About Product  \
7359  Make sure this fits by entering your model num...   
8901 